<b> Tutorial based on 6.1 Example: Learning XOR (page 170 in Deep Learning book) </b><p>
The XOR function (“exclusive or”): operation on two binary values, x1 and x2.
When only one of these values==1, the XOR function returns 1. Otherwise, 0.
Right now, not concerned with statistical generalization. 
We want our network to perform correctly on the four points X = {[0,0] , [0,1] , [1,0] , and [1,1] }.<p>
We can treat this problem as a regression problem and use a mean squared error loss function to simplify the math for this example as much as possible (there are better approaches for modeling binary data).

In [1]:
#load tensorflow
import tensorflow as tf

<b>Network 1 - Single Layer </b><p>We can minimize in closed form with respect to w and b using the normal equations.
After solving the normal equations, we obtain w = 0 and b = 1/2. 
The linear model simply outputs 0.5 everywhere.

In [2]:
##Network 1##
sess1 = tf.Session()
#input data
XOR_X = [[0,0],[0,1],[1,0],[1,1]] #input

#placeholders
x_ = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
#use weights/biases from book solution (page 171)
w = tf.Variable(tf.zeros([2,1]), tf.float32)
b = tf.Variable([1/2.], tf.float32)

init = tf.global_variables_initializer()
sess1.run(init)

#operation node
linear_model = tf.matmul(x_,w) + b 

#see what the predictions are
print(sess1.run(linear_model, {x_: XOR_X})) 

[[ 0.5]
 [ 0.5]
 [ 0.5]
 [ 0.5]]


<b>Network 2 - Two Layers </b><p>
Solve the same problem using a model that learns a different feature space in which a linear model is able to represent the solution.
Introduce a very simple feedforward network with one hidden layer containing two hidden units -> change what is given to output layer.

In [3]:
##Network 2##
sess2 = tf.Session()
#input data
XOR_X = [[0,0],[0,1],[1,0],[1,1]] #input

#placeholders
x_ = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
#use weights/biases from book example (page 173)
w1 = tf.Variable(tf.ones([2,2]), tf.float32) #W
w2 = tf.Variable([[1.],[-2.]], tf.float32) #w
b1 = tf.Variable([[0.,-1.]], tf.float32) #c
b2 = tf.Variable(tf.zeros(1), tf.float32) #b

init2 = tf.global_variables_initializer()
sess2.run(init2)

#operation nodes
transformedH = tf.nn.relu(tf.matmul(x_,w1) + b1, name=None) #hidden layer with rect. linear act. func.
linear_model = tf.matmul(transformedH, w2) + b2

#see what the predictions are
print(sess2.run(linear_model, {x_: XOR_X})) 

[[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]


<b>Network 3 - Two Layers + Optimization w/random initial param weights</b><p>
In a real situation, there are lots of model parameters and training examples, 
we cannot guess the solution as we did above. Instead, a gradient-based optimization algorithm can 
find parameters that produce very little error. <p>
Now solve the same problem but let's use gradient-based optimization to find params 
in order to do so need to measure error/loss (also need predicted values!).

In [4]:
##Network 3##
sess3 = tf.Session()
#input data
XOR_X = [[0,0],[0,1],[1,0],[1,1]] #input
XOR_Y = [[0],[1],[1],[0]] #predicted

#placeholders, now we need one for predicted vals too
x_ = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
y_ = tf.placeholder(tf.float32, shape=[4,1], name="y-input") 
#now we will define with some random values as starting points 
w1 = tf.Variable(tf.random_uniform([2,2], -2, 2), tf.float32) #W
w2 = tf.Variable(tf.random_uniform([2,1], -2, 2), tf.float32) #w
b1 = tf.Variable(tf.zeros([2]), tf.float32) #c
b2 = tf.Variable(tf.zeros([1]), tf.float32) #b

#operation nodes
transformedH = tf.nn.relu(tf.matmul(x_,w1) + b1) #hidden layer with rect. linear act. func.
linear_model = tf.matmul(transformedH, w2) + b2

#MSE
loss = tf.reduce_sum(tf.square(linear_model - y_)) #create error vector.We call tf.square to square that error.

#gradient descent 
optimizer = tf.train.GradientDescentOptimizer(0.01) #0.01 is learning rate
train = optimizer.minimize(loss) #feed optimizer loss function 

init3 = tf.global_variables_initializer()
sess3.run(init3)

#train it
for i in range(10000):
        sess3.run(train, {x_: XOR_X, y_: XOR_Y})

#take a look at the results
predictions = sess3.run(linear_model, {x_: XOR_X}) 
curr_w1, curr_w2, curr_b1, curr_b2, curr_loss  = sess3.run([w1, w2, b1, b2, loss], {x_: XOR_X, y_: XOR_Y})
hidlay  = sess3.run(transformedH, {x_: XOR_X, y_: XOR_Y})
print("predictions:\n %s\n hlayout:\n %s\n"%(predictions,hidlay))
print("w1:\n %s \nw2:\n %s \nb1: %s \nb2: %s \nloss: %s"%(curr_w1, curr_w2, curr_b1, curr_b2, curr_loss))

predictions:
 [[  2.73580895e-06]
 [  9.99998331e-01]
 [  9.99998331e-01]
 [  5.58800139e-07]]
 hlayout:
 [[  2.82178867e-08   0.00000000e+00]
 [  9.03214455e-01   0.00000000e+00]
 [  9.03214455e-01   0.00000000e+00]
 [  1.80642891e+00   9.32806492e-01]]

w1:
 [[ 0.90321445  0.93280649]
 [ 0.90321445  0.93280649]] 
w2:
 [[ 1.10715199]
 [-2.14406037]] 
b1: [  2.82178867e-08  -9.32806492e-01] 
b2: [  2.70456735e-06] 
loss: 1.33676e-11


<b>Network 4 - Two Layers + Optimization w/non-random initial param weights </b><p>
Using the approach above we will often find a different solution because the minima found depends 
on the rand. initial weights (if sess3 ran enough, will find similar solution 2 examples every once in a while).
If we set the weights closer to the values provided in the example we consistently get the same results.

In [5]:
##Network 4##
sess4 = tf.Session()
#input data
XOR_X = [[0,0],[0,1],[1,0],[1,1]] #input
XOR_Y = [[0],[1],[1],[0]] #predicted

#placeholders, now we need one for predicted vals too
x_ = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
y_ = tf.placeholder(tf.float32, shape=[4,1], name="y-input") 
#constrain rand. values
w1 = tf.Variable(tf.random_uniform([2,2], .7, 1.3), tf.float32) #W
w2 = tf.Variable(tf.random_uniform([2,1], -2, 1), tf.float32) #w
b1 = tf.Variable(tf.zeros([2]), tf.float32) #c
b2 = tf.Variable(tf.zeros([1]), tf.float32) #b

#operation nodes
transformedH = tf.nn.relu(tf.matmul(x_,w1) + b1) #hidden layer with rect. linear act. func.
linear_model = tf.matmul(transformedH, w2) + b2

#MSE
loss = tf.reduce_sum(tf.square(linear_model - y_)) #create error vector.We call tf.square to square that error.

#gradient descent 
optimizer = tf.train.GradientDescentOptimizer(0.01) #0.01 is learning rate
train = optimizer.minimize(loss) #feed optimizer loss function 

init4 = tf.global_variables_initializer()
sess4.run(init4)

#train it
for i in range(10000):
        sess4.run(train, {x_: XOR_X, y_: XOR_Y})

#stake a look at the results
predictions = sess4.run(linear_model, {x_: XOR_X}) 
curr_w1, curr_w2, curr_b1, curr_b2, curr_loss  = sess4.run([w1, w2, b1, b2, loss], {x_: XOR_X, y_: XOR_Y})
hidlay  = sess4.run(transformedH, {x_: XOR_X, y_: XOR_Y})
print("predictions:\n %s\n hlayout:\n %s\n"%(predictions,hidlay))
print("w1:\n %s \nw2:\n %s \nb1: %s \nb2: %s \nloss: %s"%(curr_w1, curr_w2, curr_b1, curr_b2, curr_loss))

predictions:
 [[  2.45183492e-06]
 [  9.99998093e-01]
 [  9.99998093e-01]
 [  1.36286235e-06]]
 hlayout:
 [[  1.96318073e-08   0.00000000e+00]
 [  1.22018981e+00   0.00000000e+00]
 [  1.22018981e+00   0.00000000e+00]
 [  2.44037962e+00   1.06932676e+00]]

w1:
 [[ 1.22018981  1.06932676]
 [ 1.22018981  1.06932676]] 
w2:
 [[ 0.81954104]
 [-1.87032855]] 
b1: [  1.96318073e-08  -1.06932676e+00] 
b2: [  2.43574596e-06] 
loss: 1.51448e-11


<b>Is there a better way to save out diagnostic info? </b><p>
The network below includes several summary ops and saves the graph so we can take a closer look in TensorBoard.
Once the network has run and event files have been generated in your log directory enter the code below into a temrinal to launch TensorBoard: <p>
tensorboard --logdir=/path/to/logdir <p>
You should get a message that looks like this (paste the link in your internet browser):<p>
Starting TensorBoard 41 on port 6006 
(You can navigate to http://168.150.16.155:6006)

In [11]:
#Clear the default graph stack and reset the global default graph.
tf.reset_default_graph()

import time

log_dir = "/Users/bmk/Google Drive/desktopBackup/PSC211_S17/tensorBoardFiles/NSC211_BKlecture_code"#"./tesnorBoardFiles/NSC211_BKlecture_code" 
start_time = time.time() #record start time later 

#input data
XOR_X = [[0,0],[0,1],[1,0],[1,1]] #input
XOR_Y = [[0],[1],[1],[0]] #predicted

#placeholders, now we need one for predicted vals too
x_ = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
y_ = tf.placeholder(tf.float32, shape=[4,1], name="y-input") 
#constrain rand. values
w1 = tf.Variable(tf.random_uniform([2,2], .7, 1.3), tf.float32,name="L1_weights") #W
w2 = tf.Variable(tf.random_uniform([2,1], -2, 1), tf.float32,name="L2_weights") #w
b1 = tf.Variable(tf.zeros([2]), tf.float32,name="L1_biases") #c
b2 = tf.Variable(tf.zeros([1]), tf.float32,name="L2_biases") #b

#add summary histograms
tf.summary.histogram('layer1_weights',w1)
tf.summary.histogram('layer2_weights',w2)
tf.summary.histogram('layer1_biases',b1)
tf.summary.histogram('layer2_biases',b2)

#operation nodes
transformedH = tf.nn.relu(tf.matmul(x_,w1) + b1) #hidden layer with rect. linear act. func.
tf.summary.histogram('transformed_output',transformedH)

linear_model = tf.matmul(transformedH, w2) + b2
tf.summary.histogram("predicted", linear_model)

#MSE
loss = tf.reduce_sum(tf.square(linear_model - y_)) #create error vector.We call tf.square to square that error.
tf.summary.scalar("curr_loss", loss)

#gradient descent 
optimizer = tf.train.GradientDescentOptimizer(0.01) #0.01 is learning rate
train = optimizer.minimize(loss) #feed optimizer loss function 

# Build the summary Tensor based on the TF collection of summaries.
summary = tf.summary.merge_all()

init = tf.global_variables_initializer()
sess = tf.Session()

# Instantiate a SummaryWriter to output summaries and the Graph.
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)

sess.run(init)

#train it
for i in range(10000):
        fdict = {x_: XOR_X, y_: XOR_Y}
        #run the session and get loss and summary info
        _, curr_loss, suminfo = sess.run([train, loss, summary], feed_dict=fdict)
        duration = time.time() - start_time
        # Write the summaries and print an overview every 100 trials.
        if i % 100 == 0:
            # Print status to stdout.
            print("Step %d: loss = %.2f (%.3f sec)" % (i, curr_loss, duration))
            # Update the events file.
            summary_writer.add_summary(suminfo, i)
            summary_writer.flush()

#take a look at the results
predictions = sess.run(linear_model, {x_: XOR_X}) 
curr_w1, curr_w2, curr_b1, curr_b2, curr_loss  = sess.run([w1, w2, b1, b2, loss], {x_: XOR_X, y_: XOR_Y})
hidlay  = sess.run(transformedH, {x_: XOR_X, y_: XOR_Y})
print("predictions:\n %s\n hlayout:\n %s\n"%(predictions,hidlay))
print("w1:\n %s \nw2:\n %s \nb1: %s \nb2: %s \nloss: %s"%(curr_w1, curr_w2, curr_b1, curr_b2, curr_loss))

Step 0: loss = 8.68 (0.351 sec)
Step 100: loss = 0.15 (0.411 sec)
Step 200: loss = 0.05 (0.473 sec)
Step 300: loss = 0.01 (0.540 sec)
Step 400: loss = 0.00 (0.610 sec)
Step 500: loss = 0.00 (0.672 sec)
Step 600: loss = 0.00 (0.735 sec)
Step 700: loss = 0.00 (0.797 sec)
Step 800: loss = 0.00 (0.868 sec)
Step 900: loss = 0.00 (0.937 sec)
Step 1000: loss = 0.00 (0.987 sec)
Step 1100: loss = 0.00 (1.042 sec)
Step 1200: loss = 0.00 (1.105 sec)
Step 1300: loss = 0.00 (1.160 sec)
Step 1400: loss = 0.00 (1.215 sec)
Step 1500: loss = 0.00 (1.270 sec)
Step 1600: loss = 0.00 (1.333 sec)
Step 1700: loss = 0.00 (1.389 sec)
Step 1800: loss = 0.00 (1.448 sec)
Step 1900: loss = 0.00 (1.502 sec)
Step 2000: loss = 0.00 (1.564 sec)
Step 2100: loss = 0.00 (1.619 sec)
Step 2200: loss = 0.00 (1.674 sec)
Step 2300: loss = 0.00 (1.732 sec)
Step 2400: loss = 0.00 (1.795 sec)
Step 2500: loss = 0.00 (1.850 sec)
Step 2600: loss = 0.00 (1.904 sec)
Step 2700: loss = 0.00 (1.959 sec)
Step 2800: loss = 0.00 (2.020 se